In [ ]:
import os
import random
import shutil
image_files_list = []


sourceImgPath = '/content/drive/MyDrive/Dataset/'
destImgPath = '/content/drive/MyDrive/Dataset_'

for root, dirs, files in os.walk(sourceImgPath):
    for file in files:
        if file.endswith(".jpeg"):
            image_files_list.append(os.path.join(root, file))


file_count = len(image_files_list)
print (file_count)

# print image_files_list
filesToCopy = random.sample(image_files_list, 500)


print(len(filesToCopy))


# iterate over all random files and move them
for file in filesToCopy:
    shutil.copy(file, destImgPath)


2000
500


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pathlib
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2

data_dir = pathlib.Path('/content/drive/MyDrive/Dataset_500')
img_height = 224
img_width = 224
IMG_SIZE = 224


In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.3,
    subset='training',
    seed=123,
    image_size = (img_height,img_width),batch_size=32
)

Found 2000 files belonging to 4 classes.
Using 1400 files for training.


In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset='training',
    seed=123,
    image_size = (img_height,img_width),batch_size=32
)

Found 2000 files belonging to 4 classes.
Using 1600 files for training.


In [ ]:
class_names = train_ds.class_names
class_names

['CNV', 'DME', 'DRUSEN', 'NORMAL']

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

**SELF** **ATTETNTION**

In [ ]:
class SelfAttention(tf.keras.layers.Layer):
    def __init__(self, channels):
        super(SelfAttention, self).__init__()
        self.channels = channels
        self.q = tf.keras.layers.Conv2D(channels // 8, 1, activation='relu', padding='same')
        #channels // 8 expression calculates the number of output channels by dividing the channels
        #variable by 8 using integer division. This expression is used to reduce the dimensionality of the feature maps in
        #subsequent layers of the self-attention module.
        self.k = tf.keras.layers.Conv2D(channels // 8, 1, activation='relu', padding='same')
        self.v = tf.keras.layers.Conv2D(channels // 2, 1, activation='relu', padding='same')
        self.o = tf.keras.layers.Conv2D(channels, 1, activation='relu', padding='same')

    def call(self, inputs):
        q = self.q(inputs)
        k = self.k(inputs)
        v = self.v(inputs)

        q = tf.reshape(q, [-1, tf.shape(q)[1] * tf.shape(q)[2], self.channels // 8])
        k = tf.reshape(k, [-1, tf.shape(k)[1] * tf.shape(k)[2], self.channels // 8])
        v = tf.reshape(v, [-1, tf.shape(v)[1] * tf.shape(v)[2], self.channels // 2])

        attention = tf.matmul(q, tf.transpose(k, [0, 2, 1]))
        attention = tf.nn.softmax(attention, axis=-1)

        attention_out = tf.matmul(attention, v)
        attention_out = tf.reshape(attention_out, [-1, tf.shape(inputs)[1], tf.shape(inputs)[2], self.channels // 2])

        out = self.o(attention_out)
        out = tf.keras.layers.add([out, inputs])

        return out



**INCEPTIONV3**

In [ ]:
import tensorflow as tf
from tensorflow import keras
print(tf.__version__)

# load pre-trained model graph, don't add final layer
base_model = tf.keras.applications.InceptionV3(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                      weights='imagenet')
#freeze the first 10 layers
for layer in base_model.layers[:10]:
  layer.trainable = False
x = base_model.output
feature_maps = base_model.output
print("feature_maps.shape",feature_maps.shape)
attention_maps = SelfAttention(channels=feature_maps.shape[-1])(feature_maps)
print("attention_maps.shape",attention_maps.shape)
# add global pooling just like in InceptionV3
new_output = tf.keras.layers.GlobalAveragePooling2D()(attention_maps)
# add new dense layer for our labels
#new_output = tf.keras.layers.Dense(N_CLASSES, activation='softmax')(new_output)
new_output = tf.keras.layers.Dense(512, activation='relu')(new_output)
new_output = tf.keras.layers.Dropout(0.4)(new_output)
predictions = tf.keras.layers.Dense(4, activation='softmax')(new_output) # this layer used for ensemble learning
model1 = tf.keras.Model(inputs= base_model.inputs, outputs=new_output)


2.12.0
87910968/87910968 [==============================] - 5s 0us/step
feature_maps.shape (None, 5, 5, 2048)
attention_maps.shape (None, 5, 5, 2048)


In [ ]:
model_filepath = '/content/drive/MyDrive/capstone/model1/model_{epoch:02d}-{accuracy:.4f}.tf'

model1.compile(loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(1e-2),
              metrics=['accuracy'])
checkpoint1 = tf.keras.callbacks.ModelCheckpoint(filepath=model_filepath,
                                                monitor='val_accuracy',mode='max',save_freq='epoch',
                                                verbose=1)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
history1= model1.fit(train_ds, validation_data=val_ds, epochs=25, callbacks=[stop_early,checkpoint1])





Epoch 1/25
44/44 [==============================] - ETA: 0s - loss: 9.0166 - accuracy: 0.2043
Epoch 1: saving model to /content/drive/MyDrive/capstone/model1/model_01-0.2043.tf


44/44 [==============================] - 126s 2s/step - loss: 9.0166 - accuracy: 0.2043 - val_loss: 5.4158 - val_accuracy: 0.2488
Epoch 2/25
44/44 [==============================] - ETA: 0s - loss: 9.1842 - accuracy: 0.2050
Epoch 2: saving model to /content/drive/MyDrive/capstone/model1/model_02-0.2050.tf


44/44 [==============================] - 54s 1s/step - loss: 9.1842 - accuracy: 0.2050 - val_loss: 5.4158 - val_accuracy: 0.2488
Epoch 3/25
44/44 [==============================] - ETA: 0s - loss: 9.6585 - accuracy: 0.1793
Epoch 3: saving model to /content/drive/MyDrive/capstone/model1/model_03-0.1793.tf


44/44 [==============================] - 53s 1s/step - loss: 9.6585 - accuracy: 0.1793 - val_loss: 5.4158 - val_accuracy: 0.2488
Epoch 4/25
44/44 [==============================] - ETA: 0s - loss: 9.3493 - accuracy: 0.1950
Epoch 4: saving model to /content/drive/MyDrive/capstone/model1/model_04-0.1950.tf


44/44 [==============================] - 54s 1s/step - loss: 9.3493 - accuracy: 0.1950 - val_loss: 5.4158 - val_accuracy: 0.2488
Epoch 5/25
44/44 [==============================] - ETA: 0s - loss: 9.1454 - accuracy: 0.1936
Epoch 5: saving model to /content/drive/MyDrive/capstone/model1/model_05-0.1936.tf


...

44/44 [==============================] - 56s 1s/step - loss: 9.2583 - accuracy: 0.2064 - val_loss: 5.4158 - val_accuracy: 0.2488
Epoch 11/25
44/44 [==============================] - ETA: 0s - loss: 9.3185 - accuracy: 0.1864
Epoch 11: saving model to /content/drive/MyDrive/capstone/model1/model_11-0.1864.tf


44/44 [==============================] - 61s 1s/step - loss: 9.3185 - accuracy: 0.1864 - val_loss: 5.4158 - val_accuracy: 0.2488
Epoch 12/25
44/44 [==============================] - ETA: 0s - loss: 9.2389 - accuracy: 0.2029
Epoch 12: saving model to /content/drive/MyDrive/capstone/model1/model_12-0.2029.tf


44/44 [==============================] - 61s 1s/step - loss: 9.2389 - accuracy: 0.2029 - val_loss: 5.4158 - val_accuracy: 0.2488
Epoch 13/25
44/44 [==============================] - ETA: 0s - loss: 9.1090 - accuracy: 0.1964
Epoch 13: saving model to /content/drive/MyDrive/capstone/model1/model_13-0.1964.tf


44/44 [==============================] - 60s 1s/step - loss: 9.1090 - accuracy: 0.1964 - val_loss: 5.4158 - val_accuracy: 0.2488
Epoch 14/25
44/44 [==============================] - ETA: 0s - loss: 9.3852 - accuracy: 0.1950
Epoch 14: saving model to /content/drive/MyDrive/capstone/model1/model_14-0.1950.tf


44/44 [==============================] - 61s 1s/step - loss: 9.3852 - accuracy: 0.1950 - val_loss: 5.4158 - val_accuracy: 0.2488


**Xception**






In [ ]:

# load pre-trained model graph, don't add final layer
base_model = tf.keras.applications.Xception(include_top=False, input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                      weights='imagenet')
#freeze the first 10 layers
for layer in base_model.layers[:10]:
  layer.trainable = False #not training base model only training top layers so not traiing from scratch
feature_maps = base_model.output
print("feature_maps.shape",feature_maps.shape)
attention_maps = SelfAttention(channels=feature_maps.shape[-1])(feature_maps)
print("attention_maps.shape",attention_maps.shape)
# add global pooling just like in InceptionV3
new_output = tf.keras.layers.GlobalAveragePooling2D()(attention_maps)
# add new dense layer for our labels
#new_output = tf.keras.layers.Dense(N_CLASSES, activation='softmax')(new_output)
new_output = tf.keras.layers.Dense(512, activation='relu')(new_output)
new_output = tf.keras.layers.Dropout(0.3)(new_output)
predictions = tf.keras.layers.Dense(4, activation='softmax')(new_output) # this layer used for ensemble learning
model3 = tf.keras.Model(inputs= base_model.inputs, outputs=new_output)


94765736/94765736 [==============================] - 1s 0us/step
feature_maps.shape (None, 7, 7, 2048)
attention_maps.shape (None, 7, 7, 2048)


In [ ]:
model_filepath = '/content/drive/MyDrive/capstone/model3/model_{epoch:02d}-{accuracy:.4f}.tf'

model3.compile(loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(1e-2),
              metrics=['accuracy'])
checkpoint3 = tf.keras.callbacks.ModelCheckpoint(filepath=model_filepath,
                                                monitor='val_accuracy',mode='max',save_freq='epoch',
                                                verbose=1)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
history3= model3.fit(train_ds, validation_data=val_ds, epochs=25, callbacks=[stop_early,checkpoint3])





Epoch 1/25
44/44 [==============================] - ETA: 0s - loss: 10.0856 - accuracy: 0.2514 
Epoch 1: saving model to /content/drive/MyDrive/capstone/model3/model_01-0.2514.tf


44/44 [==============================] - 1576s 35s/step - loss: 10.0856 - accuracy: 0.2514 - val_loss: 8.7321 - val_accuracy: 0.2475
Epoch 2/25
44/44 [==============================] - ETA: 0s - loss: 9.6226 - accuracy: 0.2564 
Epoch 2: saving model to /content/drive/MyDrive/capstone/model3/model_02-0.2564.tf


44/44 [==============================] - 1548s 35s/step - loss: 9.6226 - accuracy: 0.2564 - val_loss: 8.7321 - val_accuracy: 0.2475
Epoch 3/25
44/44 [==============================] - ETA: 0s - loss: 9.6979 - accuracy: 0.2536 
Epoch 3: saving model to /content/drive/MyDrive/capstone/model3/model_03-0.2536.tf


44/44 [==============================] - 1469s 34s/step - loss: 9.6979 - accuracy: 0.2536 - val_loss: 8.7321 - val_accuracy: 0.2475
Epoch 4/25
44/44 [==============================] - ETA: 0s - loss: 9.8874 - accuracy: 0.2443 
Epoch 4: saving model to /content/drive/MyDrive/capstone/model3/model_04-0.2443.tf


44/44 [==============================] - 1436s 33s/step - loss: 9.8874 - accuracy: 0.2443 - val_loss: 8.7321 - val_accuracy: 0.2475
Epoch 5/25
44/44 [==============================] - ETA: 0s - loss: 9.9410 - accuracy: 0.2507 

**LOADING THE TRAINED MODELS**

In [ ]:
model_1= tf.keras.models.load_model('/content/drive/MyDrive/capstone/model1/model_10-0.2064.tf')
model_1= tf.keras.models.Model(inputs = model_1.inputs, outputs = model_1.outputs, name ='name_of_model1')

In [ ]:
model_2= tf.keras.models.load_model('/content/drive/MyDrive/capstone/model2/-01-0.0000.tf')
model_2= tf.keras.models.Model(inputs = model_2.inputs, outputs = model_2.outputs, name ='name_of_model2')


**TRAIN ENSEMBLE**

In [ ]:
model_filepath = '/content/drive/MyDrive/capstone/ensemble_model/-{epoch:02d}-{val_accuracy:.4f}.tf'
checkpoint_final = tf.keras.callbacks.ModelCheckpoint( filepath=model_filepath,
                                                monitor='val_accuracy',mode='max',save_best_only=True,
                                                verbose=1)

In [ ]:
models = [model_1, model_2]
model_input = tf.keras.layers.Input(shape=(224,224,3))
model_outputs = [model(model_input) for model in models]
ensemble_output = tf.keras.layers.Average()(model_outputs)
ensemble_model = tf.keras.models.Model(inputs = model_input, outputs= ensemble_output, name='ensemble')

In [ ]:
model_filepath = '/content/drive/MyDrive/capstone/ENSEMBLE/model_{epoch:02d}-{accuracy:.4f}.tf'

ensemble_model.compile(loss=tf.losses.SparseCategoricalCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(1e-2),
              metrics=['accuracy'])
checkpoint_ = tf.keras.callbacks.ModelCheckpoint(filepath=model_filepath,
                                                monitor='val_accuracy',mode='max',save_freq='epoch',
                                                verbose=1)
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
history_= ensemble_model.fit(train_ds, validation_data=val_ds, epochs=25, callbacks=[stop_early,checkpoint_])

In [ ]:
acc = history_.history['accuracy']
val_acc = history_.history['val_accuracy']
loss = history_.history['loss']
val_loss = history_.history['val_loss']

epochs = range(len(acc))
plt.plot(epochs, acc, 'b',label='Training accuracy')
plt.plot(epochs, val_acc, 'r',label='Validation accuracy')
plt.title('Training and Validation Acuuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'b',label='Training Loss')
plt.plot(epochs, val_loss, 'r',label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.figure()